In [1]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr
import esmtools

In [2]:
from dask.distributed import Client
c = Client()
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42005 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:42005/status,
Dashboard: http://127.0.0.1:42005/status,Workers: 5
Total threads: 20,Total memory: 40.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34701,Workers: 5
Dashboard: http://127.0.0.1:42005/status,Total threads: 20
Started: Just now,Total memory: 40.00 GiB
Comm: tcp://127.0.0.1:40073,Total threads: 4
Dashboard: http://127.0.0.1:39737/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:42041,


# load data

In [3]:
chunk_size = {"x":133,"y":320}

## halosteric

In [4]:
path = '/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/una12nf/'

decomposition = 'halosteric'

nexp = 'GAI'
sl_GAI = xr.open_dataset(path+nexp+'/'+decomposition+'/'+'eORCA025.L75-IMHOTEP.'+nexp+'_y2005_2018.1y_halosteric_0_bottomprime.nc',chunks=chunk_size).sosteric
nexp = 'AI'
sl_AI = xr.open_dataset(path+nexp+'/'+decomposition+'/'+'eORCA025.L75-IMHOTEP.'+nexp+'_y2005_2018.1y_halosteric_0_bottomprime.nc',chunks=chunk_size).sosteric
nexp = 'S'
sl_S = xr.open_dataset(path+nexp+'/'+decomposition+'/'+'eORCA025.L75-IMHOTEP.'+nexp+'_y2005_2018.1y_halosteric_0_bottomprime.nc',chunks=chunk_size).sosteric

# GAI

In [6]:
sl_GAI_comp = (sl_GAI*1000).compute()


In [7]:
%%time
lin_GAI = esmtools.stats.linregress(sl_GAI_comp, dim="year")

/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/esmtools/stats.py:394: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  results = xr.apply_ufunc(


CPU times: user 8min 50s, sys: 42.2 s, total: 9min 32s
Wall time: 8min 48s


In [8]:
diro = "/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/una12nf/"

# convert into dataset and save : GAI
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_GAI.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="halosteric - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 16 Feb 2024"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_halosteric_0_bottom_trendGAI_linregress.nc")


# DIFF

In [5]:
%%time
sl_GAI_AI = ( (sl_GAI - sl_AI)*1000).compute()
sl_AI_S   = ( (sl_AI  - sl_S) *1000).compute()
sl_GAI_S  = ( (sl_GAI - sl_S) *1000).compute()

CPU times: user 760 ms, sys: 531 ms, total: 1.29 s
Wall time: 2.85 s


In [6]:
%%time
lin_GAI_AI = esmtools.stats.linregress(sl_GAI_AI, dim="year")
lin_AI_S   = esmtools.stats.linregress(sl_AI_S, dim="year")
lin_GAI_S  = esmtools.stats.linregress(sl_GAI_S, dim="year")

/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/esmtools/stats.py:394: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  results = xr.apply_ufunc(


CPU times: user 33min 3s, sys: 2min 37s, total: 35min 41s
Wall time: 33min 3s


In [7]:
%%time

diro = "/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/una12nf/"

# convert into dataset and save : GAI -AI
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_GAI_AI.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="SSH - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 13 Nov 2023"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_halosteric_0_bottom_trendGAI_AI_linregress.nc")


# convert into dataset and save : AI -S
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_AI_S.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="SSH - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 13 Nov 2023"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_halosteric_0_bottom_trendAI_S_linregress.nc")


# convert into dataset and save : GAI -S
ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_GAI_S.data)),
                    coords=dict(parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
            attrs=dict(description="SSH - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 13 Nov 2023"))

ds2save.to_netcdf(diro + "/eORCA025.L75-IMHOTEP._y2005_2018.1y_halosteric_0_bottom_trendGAI_S_linregress.nc")

CPU times: user 39 ms, sys: 71.3 ms, total: 110 ms
Wall time: 269 ms
